<a href="https://colab.research.google.com/github/mokbulsrk06/Anveshan-Hackthon-2025/blob/main/Copy_of_real_or_fake_job_(95_8_acc_on_test).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding,SimpleRNN,LSTM,GRU,Dense
from tensorflow.keras.optimizers import Adam

In [ ]:
from google.colab import files
uploaded = files.upload()  # upload your CSV

df.head()

In [ ]:
df.info()

In [ ]:
df.isna().sum()


In [ ]:
df.drop(columns= ['job_id', 'salary_range'], inplace= True)


In [ ]:
df.fillna(" ",inplace = True)

In [ ]:
df['text'] = df['title'] + ' ' + df['location'] + ' ' + df['department'] + ' ' + df['company_profile'] + ' ' + df['description'] + ' ' + df['requirements'] + ' ' + df['benefits'] + ' ' + df['employment_type'] + ' ' + df['required_education'] + ' ' + df['industry'] + ' ' + df['function']

In [ ]:
df = df[["text", "telecommuting", "has_company_logo", "has_questions", "fraudulent"]]


In [ ]:
df.head()

In [ ]:
df["fraudulent"].value_counts()

In [ ]:
Tokenizer = Tokenizer()
Tokenizer.fit_on_texts(df['text'])

In [ ]:
Tokenizer.word_index

In [ ]:
sequences= Tokenizer.texts_to_sequences(df['text'])

In [ ]:
sequences[0]

In [ ]:
max_number=max([len(x) for x in sequences])
max_number

In [ ]:
pad=pad_sequences(sequences, maxlen=max_number, padding='pre', truncating='post')
pad[0]

In [ ]:
x_train, x_temp, y_train, y_temp = train_test_split(
    pad,df["fraudulent"] , test_size=0.4, random_state=42
)

x_val, x_test, y_val, y_test = train_test_split(
    x_temp, y_temp, test_size=0.5, random_state=42
)

In [ ]:
class_namee = np.unique(y_train)
class_namee

In [ ]:
vocab_size=len(Tokenizer.word_index)+1
vocab_size

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
early_stop = EarlyStopping(
            monitor='val_loss',
            patience=10,
            verbose=1,
            mode='min',
            restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(
            monitor='val_loss',
            factor=0.2,
            patience=3,
            min_lr=1e-30,
            verbose=1,
            mode='min')
Checkpoint= ModelCheckpoint(
            'cancer_model.keras',
            monitor='val_accuracy',
            save_best_only=True,
            verbose=1,
            mode='max')
callbacks = [early_stop, reduce_lr, Checkpoint]

In [ ]:
RNN_model=Sequential([
    Embedding(input_dim=vocab_size, output_dim=128, input_length=max_number),
    SimpleRNN(256, return_sequences=True,activation='tanh'),
    SimpleRNN(128, return_sequences=False,activation='tanh'),
    Dense(64,activation='tanh'),
    Dense(32,activation='tanh'),
    Dense(1, activation='sigmoid')
])
RNN_model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])
RNN_model.summary()

In [ ]:
RNN_model.fit(x_train, y_train,
              validation_data=(x_val, y_val),
              epochs=20, batch_size=16, verbose=1, callbacks=callbacks)

In [ ]:
RNN_model.evaluate(x_train, y_train)
RNN_model.evaluate(x_val, y_val)
RNN_model.evaluate(x_test, y_test)

In [ ]:
preds = RNN_model.predict(x_test)
y_pred = np.argmax(preds, axis=1)

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

y_true = y_test.values  # Use the values of the pandas Series
cm = confusion_matrix(y_true, y_pred)

plt.figure(figsize=(10, 8))
sns.heatmap(
    cm,
    annot=True,
    fmt='d',
    cmap="Blues",
    xticklabels=class_namee,  # Use the unique class labels
    yticklabels=class_namee)  # Use the unique class labels

plt.title('Confusion Matrix')
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.xticks(rotation=45)
plt.yticks(rotation=0)
plt.show()

In [ ]:
from sklearn.metrics import classification_report

print(classification_report(y_test, y_pred))